In [1]:
%reload_ext autoreload
%autoreload 2

In [40]:
import torch
from main import get_args_parser
from models import build_models


args = get_args_parser().parse_args(['--device', 'cpu'])
tokenizer, model, criterion = build_models(2, args)
n_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("number of params:", n_parameters)

Some weights of the model checkpoint at allenai/led-base-16384 were not used when initializing LEDModel: ['lm_head.weight', 'final_logits_bias']
- This IS expected if you are initializing LEDModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LEDModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


number of params: 163801093


In [41]:
model.set_transformer_trainable(False)
n_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("number of params:", n_parameters)

number of params: 1956613


In [51]:
optimizer = torch.optim.AdamW(
    model.last_layers_parameters(), lr=args.lr, weight_decay=args.weight_decay
)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, args.lr_drop)

In [53]:
optimizer.step()
lr_scheduler.step()

In [55]:
for group in optimizer.param_groups:
    for k, v in group.items():
        if k != 'params':
            print(k, v)
    print()

lr_scheduler.state_dict()

lr 0.0001
betas (0.9, 0.999)
eps 1e-08
weight_decay 0.0001
amsgrad False
initial_lr 0.0001



{'step_size': 200,
 'gamma': 0.1,
 'base_lrs': [0.0001],
 'last_epoch': 2,
 '_step_count': 3,
 'verbose': False,
 '_get_lr_called_within_step': False,
 '_last_lr': [0.0001]}

In [56]:
model.set_transformer_trainable(True)
n_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("number of params:", n_parameters)

optimizer.add_param_group({'params': model.transformer_parameters(), 'lr': args.transformer_lr})

number of params: 163801093


In [57]:
optimizer.step()
lr_scheduler.step()

In [58]:
for group in optimizer.param_groups:
    for k, v in group.items():
        if k != 'params':
            print(k, v)
    print()

lr_scheduler.state_dict()

lr 0.0001
betas (0.9, 0.999)
eps 1e-08
weight_decay 0.0001
amsgrad False
initial_lr 0.0001

lr 1e-05
betas (0.9, 0.999)
eps 1e-08
weight_decay 0.0001
amsgrad False



{'step_size': 200,
 'gamma': 0.1,
 'base_lrs': [0.0001],
 'last_epoch': 3,
 '_step_count': 4,
 'verbose': False,
 '_get_lr_called_within_step': False,
 '_last_lr': [0.0001, 1e-05]}